In [ ]:
# !pip install imutils
# !pip install opencv-python
# https://pypi.org/simple/dlib/ 에서 .whl 파일 받은 후 !pip install dlib-19.7.0-cp36-cp36m-win_amd64.whl

In [ ]:
# import the necessary packages
from collections import OrderedDict
import numpy as np
import cv2
import argparse
import dlib
import imutils

facial_features_cordinates = {}

# define a dictionary that maps the indexes of the facial
# landmarks to specific face regions
FACIAL_LANDMARKS_INDEXES = OrderedDict([
    ("Mouth", (48, 68)),
    ("Right_Eyebrow", (17, 22)),
    ("Left_Eyebrow", (22, 27)),
    ("Right_Eye", (36, 42)),
    ("Left_Eye", (42, 48)),
    ("Nose", (27, 35)),
    ("Jaw", (0, 17))
])


def shape_to_numpy_array(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coordinates = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coordinates[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coordinates


def visualize_facial_landmarks(image, shape, colors=None, alpha=0.75):
    # create two copies of the input image -- one for the
    # overlay and one for the final output image
    overlay = image.copy()
    output = image.copy()

    # if the colors list is None, initialize it with a unique

    # color for each facial landmark region
    if colors is None:
        colors = [(19, 199, 109), (79, 76, 240), (230, 159, 23),
                  (168, 100, 168), (158, 163, 32),
                  (163, 38, 32), (180, 42, 220)] # 색 지정

    # loop over the facial landmark regions individually
    for (i, name) in enumerate(FACIAL_LANDMARKS_INDEXES.keys()):

        # grab the (x, y)-coordinates associated with the
        # face landmark
        (j, k) = FACIAL_LANDMARKS_INDEXES[name]

        pts = shape[j:k]
        facial_features_cordinates[name] = pts

        # check if are supposed to draw the jawline
        if name == "Jaw":

            # since the jawline is a non-enclosed facial region,
            # just draw lines between the (x, y)-coordinates
            for l in range(1, len(pts)):
                ptA = tuple(pts[l - 1])
                ptB = tuple(pts[l])
                cv2.line(overlay, ptA, ptB, colors[i], 2)

        # otherwise, compute the convex hull of the facial
        # landmark coordinates points and display it
        else:
            hull = cv2.convexHull(pts)
            cv2.drawContours(overlay, [hull], -1, colors[i], -1)

    # apply the transparent overlay
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)

    # return the output image
    print(facial_features_cordinates)
    return output
   
    
def modify_visualize_facial_landmarks(image, shape, colors=None, alpha=0.75):
    overlay = image.copy()
    output = image.copy()
    
    if colors is None:
        colors = [(0, 0, 0)] # 색 지정
        
    for (i, name) in enumerate(FACIAL_LANDMARKS_INDEXES.keys()):
        (j, k) = FACIAL_LANDMARKS_INDEXES[name]

        pts = shape[j:k]
        facial_features_cordinates[name] = pts
        
        if name == 'Right_Eye' or name == 'Left_Eye' or name == 'Mouth':
            cv2.rectangle(overlay, 
                          (min(list([i[0] for i in facial_features_cordinates[name]])), min(list([i[1] for i in facial_features_cordinates[name]]))),  
                          (max(list([i[0] for i in facial_features_cordinates[name]])), max(list([i[1] for i in facial_features_cordinates[name]]))), 
                          colors[0], -1)
        
        cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)
    return output
    
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector() # dlib 라이브러리의 얼굴 인식 함수
predictor = dlib.shape_predictor('./model/shape_predictor_68_face_landmarks.dat')

# load the input image, resize it, and convert it to grayscale
image = cv2.imread('images/image.PNG') # 이미지 경로

image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # 흑백 이미지 변환

# detect faces in the grayscale image
rects = detector(gray, 1) # 변환한 흑백 이미지 얼굴 인식 실행
                          # rects 변수에는 인식된 얼굴의 갯수와 얼굴의 좌, 위, 우, 하 순서의 위치 픽셀 저장

# loop over the face detections
for (i, rect) in enumerate(rects):
    shape = predictor(gray, rect) # shape_predictor_68_face_landmarks.dat 이거는 얼굴을 68개의 좌표로 return
                                  # 입, 우 눈썹, 좌 눈썹, 우 눈, 좌 눈, 코, 턱선을 총 68개의 좌표로 맵핑
    
    shape = shape_to_numpy_array(shape) # 68개의 좌표를 array 형태인 68x2의 2차원 Matrix로 변환
#     shape = cover_eye_mouth(shape)

#     output = visualize_facial_landmarks(image, shape)
    output = modify_visualize_facial_landmarks(image, shape)

    cv2.imshow("Image", output)
    cv2.waitKey(0)